In [1]:
import pandas as pd
import numpy as np

import sqlite3



In [3]:
names = ('Badges', 'PostLinks', 'Tags', 'Votes', 'Comments', 'Posts', 'Users')

for name in names:
    print(name)
    print(
        pd.read_csv(f"Exercise02/{name}.csv").head()
    )
    print('-'*100)

Badges
   Class                     Date  Id            Name  TagBased  UserId
0      3  2010-08-25T19:26:49.423   1  Autobiographer     False       2
1      3  2010-08-25T19:26:49.440   2  Autobiographer     False       3
2      3  2010-08-25T19:26:49.457   3  Autobiographer     False       4
3      3  2010-08-25T19:41:49.283   4  Autobiographer     False       6
4      3  2010-08-25T19:41:49.283   5  Autobiographer     False       7
----------------------------------------------------------------------------------------------------
PostLinks
              CreationDate  Id  LinkTypeId  PostId  RelatedPostId
0  2010-08-26T20:33:50.797  31           1     262            261
1  2010-08-29T21:51:09.093  47           1     401            244
2  2010-08-30T01:57:19.137  48           1     132              3
3  2010-08-31T19:13:55.077  65           1     367             27
4  2010-08-31T19:13:55.077  66           1     367            246
------------------------------------------------------

In [6]:
conn = sqlite3.connect('pandas_02')
dfs = {}
for name in names:
    df = pd.read_csv(f"Exercise02/{name}.csv")
    dfs[name] = df
    df.to_sql(name, con=conn, if_exists='replace')

In [7]:
query1 = """
--- Zadanie 1
SELECT Tags.TagName, Tags.Count, Posts.OwnerUserId,
Users.Location, Users.DisplayName
FROM Tags
JOIN Posts ON Posts.Id=Tags.WikiPostId
JOIN Users ON Users.AccountId=Posts.OwnerUserId
WHERE OwnerUserId != -1
ORDER BY Count DESC
"""

In [8]:
pd.read_sql_query(query1,conn)

,TagName,Count,OwnerUserId,Location,DisplayName
0,freewheel,167,3021.0,"Moscow, Russia",jfs
1,preventative-maintenance,62,7044.0,"Manchester, United Kingdom",Codebeef
2,7-speed,21,7044.0,"Manchester, United Kingdom",Codebeef
3,lbs,15,3649.0,None,Tim
4,reliability,5,3927.0,"Liverpool, United Kingdom",Will Boyce
5,sociable,2,7044.0,"Manchester, United Kingdom",Codebeef
6,ordinary,1,7044.0,"Manchester, United Kingdom",Codebeef
7,dicycle,1,7044.0,"Manchester, United Kingdom",Codebeef


In [52]:
posts_df = dfs['Posts']
users_df = dfs['Users']
tags_df = dfs['Tags']


In [53]:
(
tags_df
 .merge(posts_df[posts_df['OwnerUserId'] != -1], left_on='WikiPostId', right_on='Id')
 .merge(users_df, left_on='OwnerUserId', right_on='AccountId')
    [['TagName', 'Count', 'OwnerUserId','Location', 'DisplayName']]
 .sort_values('Count')
)

,TagName,Count,OwnerUserId,Location,DisplayName
3,ordinary,1,7044.0,"Manchester, United Kingdom",Codebeef
4,dicycle,1,7044.0,"Manchester, United Kingdom",Codebeef
5,sociable,2,7044.0,"Manchester, United Kingdom",Codebeef
7,reliability,5,3927.0,"Liverpool, United Kingdom",Will Boyce
6,lbs,15,3649.0,NaN,Tim
1,7-speed,21,7044.0,"Manchester, United Kingdom",Codebeef
2,preventative-maintenance,62,7044.0,"Manchester, United Kingdom",Codebeef
0,freewheel,167,3021.0,"Moscow, Russia",jfs


In [51]:
## nie wykoynywać to jest to samo co wyżej ale nadpisuje DF 
"""
tags_df = dfs['Tags']
posts_df = dfs['Posts']
users_df = dfs['Users']

posts_df = posts_df[posts_df['OwnerUserId'] != -1] # najpierw filtr

tags_posts_df = tags_df.merge(posts_df, left_on='WikiPostId', right_on='Id')
tags_posts_df = tags_posts_df.merge(users_df, left_on='OwnerUserId', right_on='AccountId')

(tags_posts_df
 .sort_values('Count', ascending=False)
 [['TagName', 'Count', 'OwnerUserId', 'Location', 'DisplayName']]
)
"""

,TagName,Count,OwnerUserId,Location,DisplayName
0,freewheel,167,3021.0,"Moscow, Russia",jfs
2,preventative-maintenance,62,7044.0,"Manchester, United Kingdom",Codebeef
1,7-speed,21,7044.0,"Manchester, United Kingdom",Codebeef
6,lbs,15,3649.0,NaN,Tim
7,reliability,5,3927.0,"Liverpool, United Kingdom",Will Boyce
5,sociable,2,7044.0,"Manchester, United Kingdom",Codebeef
3,ordinary,1,7044.0,"Manchester, United Kingdom",Codebeef
4,dicycle,1,7044.0,"Manchester, United Kingdom",Codebeef


In [36]:
query4 = """
--- Zadanie 4
SELECT Posts.Title, RelatedTab.NumLinks
FROM
    (SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
    FROM PostLinks GROUP BY RelatedPostId) AS RelatedTab
JOIN Posts ON RelatedTab.PostId=Posts.Id
WHERE Posts.PostTypeId=1
ORDER BY NumLinks DESC
"""

In [54]:
posts_links = dfs['PostLinks']

RelatedTab = posts_links.groupby('RelatedPostId')[['PostId']].count().reset_index()
RelatedTab = RelatedTab.rename(columns={'RelatedPostId':'PostId','PostId':'NumLinks'})
(
    RelatedTab
 .merge(posts_df[posts_df['PostTypeId']==1], left_on = 'PostId', right_on = 'Id')
 .sort_values('NumLinks', ascending=False)[['Title','NumLinks']]
)

,Title,NumLinks
42,Terminology index - a list of bike part names ...,156
2064,Why shouldn't I care what model/make/year my b...,154
1995,How can I tell what year my bike was made?,100
2077,What is the maximum or minimum tire width I ca...,42
627,Front suspension or full suspension mountain bike,37
...,...,...
406,I am about to buy a new bicycle - what should ...,1
849,Disassembly and Shipping of bike,1
1340,Rotating two chains on one cassette,1
850,I was in a collision the other day. What shoul...,1


In [ ]:
"""
postlinks_df = dfs['PostLinks']
posts_df = dfs['Posts']


related_tab_df = (postlinks_df
 .groupby('RelatedPostId')[['PostId']]
 .count()
 .reset_index()
 .rename(columns={
     'PostId': 'NumLinks',
     'RelatedPostId': 'PostId'
    })
)

(related_tab_df
 .merge(posts_df[posts_df['PostTypeId'] == 1], left_on='PostId', right_on='Id')
 [['Title', 'NumLinks']]
 .sort_values('NumLinks', ascending=False)
)
"""

UpVotesTab

|PostId| UpVotes|
|---|---|
|    1 |5|
|    2 |3|

DownVotes

|PostId| DownVotes|
|---|---|
|2|2|
     
|PostId|UpVotes|IFNULL(DownVotes, 0)|
|---|---|---|
|1|5|0|
|2|3|2|


In [55]:
query6 = """
--- Zadanie 6
SELECT 
    UpVotesTab.PostId, 
    UpVotesTab.UpVotes, 
    IFNULL(DownVotesTab.DownVotes, 0) AS DownVotes
FROM
(
    SELECT PostId, COUNT(*) AS UpVotes FROM Votes
    WHERE VoteTypeId=2 GROUP BY PostId
) AS UpVotesTab
LEFT JOIN
(
    SELECT PostId, COUNT(*) AS DownVotes FROM Votes
    WHERE VoteTypeId=3 GROUP BY PostId
) AS DownVotesTab
ON UpVotesTab.PostId=DownVotesTab.PostId
"""

In [74]:
votes_df = dfs['Votes']


In [75]:
(
    votes_df[votes_df['VoteTypeId']==2]
    .groupby('PostId')
    .count()[['Id']]
    .reset_index()
    .rename(columns={'Id':'UpVotesTab'})
    .merge(votes_df[votes_df['VoteTypeId']==3]
           .groupby('PostId')
           .count()[['Id']]
           .reset_index()
           .rename(columns={'Id':'DownVotesTab'}),
          on='PostId', how='left')
    .fillna(0)
)

,PostId,UpVotesTab,DownVotesTab
0,1,42,0.0
1,2,13,0.0
2,3,10,0.0
3,4,16,0.0
4,5,31,0.0
...,...,...,...
41776,63933,2,0.0
41777,63934,1,0.0
41778,63939,1,0.0
41779,63940,1,0.0


In [69]:
df_votes=dfs['Votes']
upvotes=df_votes[df_votes['VoteTypeId']==2]
upvotes=upvotes.groupby('PostId').count()[['Id']].reset_index().rename(columns={"Id":'upvotes'})

In [71]:
df_votes=dfs['Votes']
downvotes=df_votes[df_votes['VoteTypeId']==3]
downvotes=downvotes.groupby('PostId').count()[['Id']].reset_index().rename(columns={"Id":'downvotes'})
upvotes.head(), downvotes.head()
upvotes.merge(downvotes,  on='PostId', how="left" ).fillna(0)

,PostId,upvotes,downvotes
0,1,42,0.0
1,2,13,0.0
2,3,10,0.0
3,4,16,0.0
4,5,31,0.0
...,...,...,...
41776,63933,2,0.0
41777,63934,1,0.0
41778,63939,1,0.0
41779,63940,1,0.0
